# Importing packages

In [27]:
!pip install efficientnet_pytorch torchtoolbox
import os
import seaborn as sns
from efficientnet_pytorch import EfficientNet
import gc
import datetime
import warnings
import numpy as np
import pandas as pd
from random import randint
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchtoolbox.transform as transforms
from torch.optim.optimizer import Optimizer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, roc_auc_score
from matplotlib.pyplot import imread
from pylab import *
from albumentations import (Cutout,JpegCompression,ShiftScaleRotate,CenterCrop,RandomRotate90,Flip,VerticalFlip,Blur,RandomContrast,HueSaturationValue,RandomGamma,Rotate,RandomBrightness,HorizontalFlip, Normalize, Resize, Compose, GaussNoise,Transpose,RandomBrightnessContrast,RandomCrop)
from albumentations.pytorch import ToTensor
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, models, transforms
import time
import copy
# !pip install timm
# import timm
# import rangelars_lookahead_ralamb
# from rangelars_lookahead_ralamb import *
import lightgbm as lgb

In [2]:
def seed_all(seed=8):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  #random.seed(seed)
  torch.backends.cudnn.enabled=False
  torch.backends.cudnn.deterministic=True

seed_all(8)

In [3]:
warnings.simplefilter('ignore')
torch.manual_seed(8)
np.random.seed(8)

# Csv file

In [13]:
tc = pd.read_csv('/content/new_train.csv')
tt = pd.read_csv('/content/test.csv')
# test_csv['target']=""
# test_csv.target=0

In [14]:
print('train image count--',len(tc),'\t test image count--',len(tt))
tc.target.value_counts(normalize=True)

train image count-- 33126 	 test image count-- 10982


0    0.98237
1    0.01763
Name: target, dtype: float64

# Feature Eng


In [15]:
# tc.age_approx.loc[np.where(tc.age_approx==10.0)]= 0
# tc.age_approx.loc[np.where(tc.age_approx==15.0)]= 0
# tc.age_approx.loc[np.where(tc.age_approx==20.0)]= 0
# tc.age_approx.loc[np.where(tc.age_approx==90.0)]= 0
# tc.age_approx.loc[np.where(tc.age_approx==80.0)]= 0
# tc.age_approx.loc[np.where(tc.age_approx==85.0)]= 0
# tc.age_approx.loc[np.where(tc.age_approx==25.0)]= 1
# tc.age_approx.loc[np.where(tc.age_approx==30.0)]= 2
# tc.age_approx.loc[np.where(tc.age_approx==35.0)]= 3
# tc.age_approx.loc[np.where(tc.age_approx==40.0)]= 4
# tc.age_approx.loc[np.where(tc.age_approx==45.0)]= 5
# tc.age_approx.loc[np.where(tc.age_approx==50.0)]= 6
# tc.age_approx.loc[np.where(tc.age_approx==55.0)]= 7
# tc.age_approx.loc[np.where(tc.age_approx==60.0)]= 8
# tc.age_approx.loc[np.where(tc.age_approx==65.0)]= 9
# tc.age_approx.loc[np.where(tc.age_approx==70.0)]= 10
# tc.age_approx.loc[np.where(tc.age_approx==75.0)]= 11
tc.drop(index=[9647,22645],inplace=True)

tc.drop(columns=['image_name','diagnosis','benign_malignant','width','height','patient_code'], inplace=True)
tc.drop(index=tc.loc[tc.anatom_site_general_challenge.isna()].index,inplace=True)
tc.drop(index=tc.loc[tc.sex.isna()].index,inplace=True)
tc.drop(index=tc.loc[tc.age_approx.isna()].index,inplace=True)
#tc.anatom_site_general_challenge.fillna(value='unknown', inplace=True)
tc = pd.get_dummies(tc, columns=['sex', 'anatom_site_general_challenge'], drop_first=True)# SPLITTING DATASET INTO TRAINING AND VALIDATION SET
print('train image count--',len(tc))
tc.head()

train image count-- 32529


,patient_id,age_approx,target,tfrecord,sex_male,anatom_site_general_challenge_lower extremity,anatom_site_general_challenge_oral/genital,anatom_site_general_challenge_palms/soles,anatom_site_general_challenge_torso,anatom_site_general_challenge_upper extremity
0,IP_7279968,45.0,0,0,1,0,0,0,0,0
1,IP_3075186,45.0,0,0,0,0,0,0,0,1
2,IP_2842074,50.0,0,6,0,1,0,0,0,0
3,IP_6890425,45.0,0,0,0,0,0,0,0,0
4,IP_8723313,55.0,0,11,0,0,0,0,0,1


In [16]:
len(tc)

32529

In [17]:
tc.age_approx.value_counts()

45.0    4286
50.0    4224
55.0    3785
40.0    3531
60.0    3201
35.0    2802
65.0    2500
30.0    2318
70.0    1949
25.0    1523
75.0     974
20.0     642
80.0     416
85.0     149
15.0     132
90.0      80
10.0      17
Name: age_approx, dtype: int64

In [18]:
# tt.age_approx.loc[np.where(tt.age_approx==10.0)]= 0
# tt.age_approx.loc[np.where(tt.age_approx==15.0)]= 0
# tt.age_approx.loc[np.where(tt.age_approx==20.0)]= 0
# tt.age_approx.loc[np.where(tt.age_approx==90.0)]= 0
# tt.age_approx.loc[np.where(tt.age_approx==80.0)]= 0
# tt.age_approx.loc[np.where(tt.age_approx==85.0)]= 0
# tt.age_approx.loc[np.where(tt.age_approx==25.0)]= 1
# tt.age_approx.loc[np.where(tt.age_approx==30.0)]= 2
# tt.age_approx.loc[np.where(tt.age_approx==35.0)]= 3
# tt.age_approx.loc[np.where(tt.age_approx==40.0)]= 4
# tt.age_approx.loc[np.where(tt.age_approx==45.0)]= 5
# tt.age_approx.loc[np.where(tt.age_approx==50.0)]= 6
# tt.age_approx.loc[np.where(tt.age_approx==55.0)]= 7
# tt.age_approx.loc[np.where(tt.age_approx==60.0)]= 8
# tt.age_approx.loc[np.where(tt.age_approx==65.0)]= 9
# tt.age_approx.loc[np.where(tt.age_approx==70.0)]= 10
# tt.age_approx.loc[np.where(tt.age_approx==75.0)]= 11
tt.drop(columns=['image_name','patient_id'], inplace=True)
tt.anatom_site_general_challenge.fillna(value='torso', inplace=True)
tt = pd.get_dummies(tt, columns=['sex', 'anatom_site_general_challenge'], drop_first=True)
tt.head() 

,age_approx,sex_male,anatom_site_general_challenge_lower extremity,anatom_site_general_challenge_oral/genital,anatom_site_general_challenge_palms/soles,anatom_site_general_challenge_torso,anatom_site_general_challenge_upper extremity
0,70.0,1,0,0,0,1,0
1,40.0,1,1,0,0,0,0
2,55.0,0,0,0,0,1,0
3,50.0,0,0,0,0,1,0
4,45.0,0,1,0,0,0,0


In [19]:
len(tt)

10982

In [20]:
tt.age_approx.value_counts()

45.0    1587
40.0    1452
55.0    1265
50.0    1230
35.0    1032
60.0    1032
70.0     815
65.0     756
30.0     702
25.0     298
80.0     207
75.0     201
20.0     187
85.0     156
15.0      41
90.0      14
10.0       7
Name: age_approx, dtype: int64

# LGBM

In [ ]:
from scipy.misc import derivative

def focal_loss_lgb(y_pred, dtrain, alpha, gamma):
  a,g = alpha, gamma
  y_true = dtrain.label
  def fl(x,t):
  	p = 1/(1+np.exp(-x))
  	return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
  partial_fl = lambda x: fl(x, y_true)
  grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
  hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
  return grad, hess


def focal_loss_lgb_eval_error(y_pred, dtrain, alpha, gamma):
  a,g = alpha, gamma
  y_true = dtrain.label
  p = 1/(1+np.exp(-y_pred))
  loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
  # (eval_name, eval_result, is_higher_better)
  return 'focal_loss', np.mean(loss), False


def focal_loss_lgb_roc_score(preds, lgbDataset):
  preds = sigmoid(preds)
  #preds = [int(p>0.5) for p in preds]
  y_true = lgbDataset.get_label()
  return 'roc_auc', roc_auc_score(y_true, preds), True

In [ ]:
d_train = lgb.Dataset(tc.drop(columns=['target','patient_id']), label=tc.target)
params = {}
#params['learning_rate'] = 1e-1
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'roc_auc'
#params['sub_feature'] = 0.5
params['num_leaves'] = 256
#params['min_data'] = 50
params['max_depth'] = -1
params['feature_fraction']= 1.0
params['bagging_fraction']= 0.75
params['random_state'] = 8
params['categorical_features'] = 0,1,2,3,4,5,6
params['min_child_weight']= 0.03454472573214212
#params['device']='gpu'
#params['verbose']=True
#params['num_boost_round']=50

clf = lgb.train(params, d_train, 5000)

In [ ]:
# focal_loss = lambda x,y: focal_loss_lgb(x, y, alpha=1, gamma=2.)
# #focal_loss_eval = lambda x,y: focal_loss_lgb_eval_error(x, y, alpha=0.25, gamma=1.)
# clf = lgb.train(params, d_train, 5000, fobj=focal_loss, feval=focal_loss_lgb_roc_score)

In [ ]:
pred=clf.predict(tc.drop(columns=['target','patient_id']))

In [ ]:
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
roc_auc_score(tc['target'].values, pred)

0.7296022271145206

In [ ]:
accuracy_score(tc['target'].values, np.zeros(len(tc)))#np.round(pred))

0.9823234652156537

In [ ]:
accuracy_score(tc['target'].values, np.round(pred))

0.9823234652156537

# CROSS VAL

In [ ]:
lr_list = [3e-2,1e-2,9e-3,8e-3,7e-3,6e-3,5e-3]
nl_list = [2,4,8]
md_list = [1,2,4,8,16,32,64]
kf_pred = np.zeros(len(tt))
count=0
for lr in lr_list:
  print('~^'*20)
  for md in md_list:
    print('^^'*20)
    for nl in nl_list:
      print("LR = ",lr,"\tmin_depth = ",md,"\tnum_leaves = ",nl)
      count+=1
      skf = StratifiedKFold(n_splits=3)
      params = {}
      params['learning_rate'] = lr
      params['boosting_type'] = 'gbdt'
      params['objective'] = 'binary'
      params['metric'] = 'roc_auc'
      #params['sub_feature'] = 0.5
      params['num_leaves'] = nl
      #params['min_data'] = 50
      params['max_depth'] = md
      params['feature_fraction']= 0.75
      params['bagging_fraction']= 0.8
      params["bagging_freq"] = 1
      params['random_state'] = 8
      params["bagging_seed"]= 8
      params['min_child_weight']= 0.03454472573214212
      val_check = np.zeros(len(tc))
      #kf_prediction = np.zeros(len(tt))
      for fold, (train_idx, val_idx) in enumerate(skf.split(X=np.zeros(len(tc)), y=tc['target']), 1):
        #print("="*20, 'Fold', fold, '='*20)
        _tr = lgb.Dataset(tc.drop(columns=['target','patient_id']).iloc[train_idx], tc['target'].iloc[train_idx])
        _val = lgb.Dataset(tc.drop(columns=['target','patient_id']).iloc[val_idx], tc['target'].iloc[val_idx])
        clf = lgb.train(params, _tr, 3000, valid_sets=[_val])
        val_check[val_idx] = clf.predict(tc.drop(columns=['target','patient_id']).iloc[val_idx])
        #print(roc_auc_score(tc.iloc[val_idx]['target'].values, val_check[val_idx]))
        kf_pred += clf.predict(tt) / skf.n_splits
      rocval=roc_auc_score(tc['target'].values, val_check)
      print(rocval)
kf_pred/=count

~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LR =  0.03 	min_depth =  1 	num_leaves =  2
0.6690047922148958
LR =  0.03 	min_depth =  1 	num_leaves =  4
0.6690047922148958
LR =  0.03 	min_depth =  1 	num_leaves =  8
0.6690047922148958
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LR =  0.03 	min_depth =  2 	num_leaves =  2
0.6690047922148958
LR =  0.03 	min_depth =  2 	num_leaves =  4
0.6771112278247806
LR =  0.03 	min_depth =  2 	num_leaves =  8
0.6771112278247806
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LR =  0.03 	min_depth =  4 	num_leaves =  2
0.6690047922148958
LR =  0.03 	min_depth =  4 	num_leaves =  4
0.6754003989430457
LR =  0.03 	min_depth =  4 	num_leaves =  8
0.6675448674861417
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
LR =  0.03 	min_depth =  8 	num_leaves =  2
0.6690047922148958
LR =  0.03 	min_depth =  8 	num_leaves =  4
0.6754003989430457
LR =  0.03 	min_depth =  8 	num_leaves =  8
0.6672363261318581
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# group k fold

In [44]:
skf = KFold(n_splits=5)#GroupKFold(n_splits=5)
params = {}
params['learning_rate'] = 0.007
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'roc_auc'
#params['sub_feature'] = 0.5
params['num_leaves'] = 4
#params['min_data'] = 50
params['max_depth'] = 4
# params['feature_fraction']= 0.75
# params['bagging_fraction']= 0.8
params["bagging_freq"] = 1
params['random_state'] = 8
params["bagging_seed"]= 8
params['min_child_weight']= 0.03454472573214212

In [45]:
#val_check = np.zeros(len(tc))
###now###
val_check = tc.copy()
kf_pred = np.zeros(len(tt))
for fold, (train_idx, val_idx) in enumerate(skf.split(np.arange(15)), 1): #, groups=tc['patient_id'].tolist()), 1):
  print("="*20, 'Fold', fold, '='*20)
  _tr = lgb.Dataset(tc.drop(columns=['target','patient_id','tfrecord']).loc[tc.tfrecord.isin(train_idx)], tc['target'].loc[tc.tfrecord.isin(train_idx)])
  _val = lgb.Dataset(tc.drop(columns=['target','patient_id','tfrecord']).loc[tc.tfrecord.isin(val_idx)], tc['target'].loc[tc.tfrecord.isin(val_idx)])
  clf = lgb.train(params, _tr, 3000, valid_sets=[_val])
  val_check.target.loc[tc.tfrecord.isin(val_idx)] = clf.predict(tc.drop(columns=['target','patient_id','tfrecord']).loc[tc.tfrecord.isin(val_idx)])
  print(roc_auc_score(tc.loc[tc.tfrecord.isin(val_idx)]['target'].values, val_check.target.loc[tc.tfrecord.isin(val_idx)]))
  kf_pred += clf.predict(tt) / skf.n_splits

==================== Fold 1 ====================
0.6984045427256352
==================== Fold 2 ====================
0.6707803474055865
==================== Fold 3 ====================
0.6536089494926516


In [46]:
#val_check1=val_check
#val_check=(val_check1 + val_check2)/2

In [47]:
roc_auc_score(tc['target'].values, val_check.target)

0.6785921610140664

In [24]:
print(accuracy_score(tc['target'].values, np.zeros(len(tc))))
print(accuracy_score(tc['target'].values, np.round(val_check)))

0.9823234652156537
0.9823234652156537


# prediction


In [36]:
#preds=clf.predict(tt)
preds=kf_pred
preds

array([0.03242851, 0.00887102, 0.01210292, ..., 0.01707794, 0.02531386,
       0.01416438])

In [37]:
sub_csv = pd.read_csv('sample_submission.csv')
sub_csv['target'] = preds

In [38]:
sub_csv.head()

,image_name,target
0,ISIC_0052060,0.032429
1,ISIC_0052349,0.008871
2,ISIC_0058510,0.012103
3,ISIC_0073313,0.009465
4,ISIC_0073502,0.009402


In [39]:
sub_csv.to_csv('/content/sub_lgb_14.csv', index=False)